### Цель и план исследования

ООО «ПРОСЕПТ» — российская производственная компания, специализирующаяся
на выпуске профессиональной химии. В своей работе используют опыт ведущих
мировых производителей и сырье крупнейших химических концернов. Производство и
логистический центр расположены в непосредственной близости от Санкт-Петербурга,
откуда продукция компании поставляется во все регионы России.
Сайт: https://prosept.ru/



Заказчик производит несколько сотен различных товаров бытовой и промышленной
химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь,
занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн
площадках.

Для оценки ситуации, управления ценами и бизнесом в целом, заказчик
периодически собирает информацию о том, как дилеры продают их товар. Для этого
они парсят сайты дилеров, а затем сопоставляют товары и цены.
Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт
заказчик. Например, могут добавляться новый слова (“универсальный”,
“эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с
товарами производителя делается вручную.
Цель этого проекта - разработка решения, которое отчасти автоматизирует процесс
сопоставления товаров. Основная идея - предлагать несколько товаров заказчика,
которые с наибольшей вероятностью соответствуют размечаемому товару дилера.
Предлагается реализовать это решение, как онлайн сервис, открываемый в веб-
браузере. Выбор наиболее вероятных подсказок делается методами машинного
обучения.

Описание данных

Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые
данные:

1 marketing_dealer - список дилеров;

2 marketing_dealerprice - результат работы парсера площадок дилеров:

○ product_key - уникальный номер позиции;
○ price - цена;
○ product_url - адрес страницы, откуда собраны данные;
○ product_name - заголовок продаваемого товара;
○ date - дата получения информации;
○ dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)


3 marketing_product - список товаров, которые производит и распространяет
заказчик;

○ article - артикул товара;
○ ean_13 - код товара (см. EAN 13)
○ name - название товара;
○ cost - стоимость;
○ min_recommended_price - рекомендованная минимальная цена;
○ recommended_price - рекомендованная цена;
○ category_id - категория товара;
○ ozon_name - названиет товара на Озоне;
○ name_1c - название товара в 1C;
○ wb_name - название товара на Wildberries;
○ ozon_article - описание для Озон;
○ wb_article - артикул для Wildberries;
○ ym_article - артикул для Яндекс.Маркета;

4 marketing_productdealerkey - таблица матчинга товаров заказчика и товаров
дилеров

○ key - внешний ключ к marketing_dealerprice
○ product_id - внешний ключ к marketing_product
○ dealer_id - внешний ключ к marketing_dealer


## План работ

1. Загрузить файлы. Изучить их. Объединить при необходимости.

2. Провести исследовательский анализ данных, обработать данные. Для строковых значений удалить все непечатные символы, привести к единому языку и стилю. Провести токенизацию и лемматизацию при необходимости.

3. Подобрать модель, которая будет ранжировать подходящие варианты. Можно рассмотреть методы knn, прямое вычисление Евклидова расстояния, Манхетенское расстояние, вычисление косинусов.

4. Рассмотреть методы векторизации строк. Рассмотреть мешки слов, TF-IDF, использование предобученной сети BERT.

5. Изучить подходящие к нашей задаче метрики (MRR, MAP@K и nDCG@K).

6. Сделать генерацию фичей и попробовать решить задачу бустингом.

7. Выбрать лучшее решение по итогам.

8. Оформить документацию.

## Результаты работы

1. Файлы содержат много не нужной информации, которая вопследствии была удалена. Были объединены таблицы marketing_dealer и marketing_dealerprice для получения больше данных для обучения моделей. Однако в этих таблицах есть дубликаты по диллеру, их успешно удалили.

2. В полученных таблицах оставили только столбцы с названием товара и его id. Строки были обработаны: убрали заглавные буквы, непечатные символы, прочие символы. Также были написаны функции, которые переводят килограммы в граммы, а литры в миллилитры.

3. Для работы мы взяли в итоге расчет Эвклидова расстояния и расчет косинусов между векторами. Это обусловлено распространненостью их применению и прошедшей обкаткой.

4. Было рассмотрено больше 100 разные методов и вариантов токенизации, лемматизации и перевода в вектора. В работе можно увидеть основные результаты.

5. Что касается метрик, то подходящей метрикой для нашей задачи, которая будет учитывать положение (ранг) искомого значения в списке предложенных моделью значений нами была выбрана метрика MRR. Метрика достаточно простая, но тем не менее положение ранга в нее входит как обратная величина, поэтому чем выше эта метрика, тем больше вероятность тогоч, то искомое знаяение будет первым в сиске найденных. А именно это нам надо по условию заказчика.

6. ...

7. По итогам работы Бэкэнду была предовставлена модель:






### Загрузка приложений

In [ ]:
import re
import os
import pandas as pd
import numpy as np

!pip install faiss-cpu

!pip install sentence_transformers
import faiss

from sentence_transformers import SentenceTransformer # initialize sentence transformer model

!pip install -U transliterate
from transliterate import translit, get_available_language_codes

import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=628fcb6fac31d32115b92fff0eaf16048bb9db8b3d32f317b8d60130970dd064
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
  Using cached transliterate-1.10.2-py2.py3-none-any.whl (45 kB)


## Определим функции

In [ ]:
def replace_values_ml(value):
  # функция обрабатывает 'л'
    if ' л' in value:
        value = value.replace(' л', '000 мл')
        value = value.replace('.0', '')  # Удаление .0 при замене литров на миллилитры
        return value
    elif 'л' in value:
        pattern = r'(\d+(?:\.\d+)?)\s*л\b'  # Паттерн для нахождения числа с возможной десятичной частью и "л"
        matches = re.findall(pattern, value, flags=re.IGNORECASE)
        for match in matches:
            replacement = f"{float(match) * 1000:.0f} мл"  # Корректное заменя "л" на "000 мл" с учетом десятичной части числа
            value = re.sub(fr'({match})\s*л\b', replacement, value, flags=re.IGNORECASE)
        return value
    else:
        return value


def replace_values_kg(value):
  # функция обрабатывает 'кг'
    if ' кг' in value:
        value = value.replace(' кг', '000 г')
        value = value.replace('.0', '')  # Удаление .0 при замене литров на миллилитры
        return value
    elif 'кг' in value:
        pattern = r'(\d+(?:\.\d+)?)\s*кг\b'  # Паттерн для нахождения числа с возможной десятичной частью и "л"
        matches = re.findall(pattern, value, flags=re.IGNORECASE)
        for match in matches:
            replacement = f"{float(match) * 1000:.0f} г"  # Корректное заменя "л" на "000 мл" с учетом десятичной части числа
            value = re.sub(fr'({match})\s*кг\b', replacement, value, flags=re.IGNORECASE)
        return value
    else:
        return value

def find_rank(arr, value):
  # функция делает список рангов соответствия
  try:
    rank = arr.index(value)+1
#    rank = arr.index(value) + 1 if value in arr else 0  # Ищем индекс значения в отсортированном массиве
    return rank
  except ValueError:
    return 0


def bert_mnb(df):
  # загрудаем модель BERT
  model = SentenceTransformer('intfloat/multilingual-e5-large') # create sentence embeddings
  df_product_embeddings = model.encode(df, normalize_embeddings=True)
  return df_product_embeddings


def process_dataset(dataset, prepros, k, test):
  #  считаем евклидовы расстояния
    df=prepros(dataset)
    d = df.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(df)
    ranks = []
    for row in test:
        # Получаем ранги для каждого значения в обработанной строке
        xq = np.array([model(test.loc[row, 'product_name'])])
        D, I = index.search(xq, k)
        value=test.loc[[row,'product_id']]
        string=dataset.loc[I[0,0], 'id']
        row_array = list(map(int, string.strip("[]").split()))
        row_ranks = find_rank(row_array, value)
        ranks.append(row_ranks)
    return ranks

def calculate_average(ranks):
  # считаем MRR
    ranks_array = np.array(ranks)
    masked_ranks = np.ma.masked_equal(ranks_array, 0)
    if masked_ranks.count() == 0:
        return 0
    average = np.mean(1 / masked_ranks)
    return average

### Загрузка и обработка файла

In [ ]:
# загружаем файлы
from google.colab import drive
drive.mount('/content/gdrive')


data_product = pd.read_csv('/content/gdrive/MyDrive/marketing_product.csv', error_bad_lines=False, sep=';')

df_product=data_product

#df_product=df_product.set_index('id').sort_index(ascending=True)


Mounted at /content/gdrive


<ipython-input-8-0e324f4faf16>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_product = pd.read_csv('/content/gdrive/MyDrive/marketing_product.csv', error_bad_lines=False, sep=';')


In [ ]:
df_productdealerkey = pd.read_csv('/content/gdrive/MyDrive/marketing_productdealerkey.csv', error_bad_lines=False, sep=';')

<ipython-input-9-fd32f451bbcc>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_productdealerkey = pd.read_csv('/content/gdrive/MyDrive/marketing_productdealerkey.csv', error_bad_lines=False, sep=';')


In [ ]:
df_dealerprice = pd.read_csv('/content/gdrive/MyDrive/marketing_dealerprice.csv', error_bad_lines=False, sep=';')

<ipython-input-10-e9f83a1ab1c7>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_dealerprice = pd.read_csv('/content/gdrive/MyDrive/marketing_dealerprice.csv', error_bad_lines=False, sep=';')


In [ ]:
# df_dealer = pd.read_csv('/content/gdrive/MyDrive/marketing_dealer.csv', error_bad_lines=False, sep=';')
# df_dealer

In [ ]:
# data_test = pd.read_excel('/content/gdrive/MyDrive/test.xlsx')
# test=data_test
# test

In [ ]:
test=df_productdealerkey.merge(df_dealerprice[['product_key', 'product_name']], left_on='key', right_on='product_key', how='inner')[['product_id', 'product_name']]
test = test.drop_duplicates().reset_index(drop=True)


### EDA

In [ ]:
df_product=df_product.fillna('  ')
df_product=df_product.astype("string")

In [ ]:
#убираем столбцы
df_product.columns

Index(['Unnamed: 0', 'id', 'article', 'ean_13', 'name', 'cost',
       'recommended_price', 'category_id', 'ozon_name', 'name_1c', 'wb_name',
       'ozon_article', 'wb_article', 'ym_article', 'wb_article_td'],
      dtype='object')

In [ ]:
df_product=df_product.drop(['Unnamed: 0', 'cost','recommended_price', 'category_id', 'ozon_article',
                            'wb_article', 'ym_article', 'wb_article_td'], axis=1)


In [ ]:
df_product.columns

Index(['id', 'article', 'ean_13', 'name', 'ozon_name', 'name_1c', 'wb_name'], dtype='object')

In [ ]:
# небольшая обработка строк

names=['name', 'ozon_name', 'name_1c', 'wb_name']
transl = lambda x: translit(x, 'ru') # Изменение 'ru' на 'en' для транслитерации с русского на английский

df_product['new_name']=df_product['name']

for name in names:
  df_product[name] = (df_product[name].str.replace("\r\n", " ").str.lower().str.replace(r"[^a-zа-я0-9\s]", ""))
  df_product[name] = df_product[name].apply(replace_values_ml)
  df_product[name] = df_product[name].apply(replace_values_kg) # Замена значений в столбце 'объем' с помощью функции replace_values
  df_product[name] = df_product[name].map(transl)  # Добавление функции map для применения транслитерации к столбцу


<ipython-input-18-2d9a2b2e0670>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_product[name] = (df_product[name].str.replace("\r\n", " ").str.lower().str.replace(r"[^a-zа-я0-9\s]", ""))


In [ ]:
df_product


,id,article,ean_13,name,ozon_name,name_1c,wb_name,new_name
0,245,008-1,4680008140234.0,антисептик невымываемыйпросепт ултраконцентрат...,антисептик невымываемый для ответственных конс...,антисептик невымываемый для ответственных конс...,антисептик невымываемый для ответственных конс...,Антисептик невымываемыйPROSEPT ULTRAконцентрат...
1,3,242-12,,антигололед 32 просептготовый состав 12000 г,,антигололед 32 просептготовый состав 12000 г,,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,443,0024-06 с,4680008145208.0,герметик акриловый цвет сосна фп 600мл,герметик акриловый для швов для деревянных дом...,герметик акриловый цвет сосна фп 600мл,герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл"
3,147,305-2,4610093420164.0,кондиционер для белья с ароматом королевского...,кондиционер для белья королевский ирис просепт...,кондиционер для белья королевский ирис просепт...,кондиционер для белья королевский ирис просепт...,Кондиционер для белья с ароматом королевского...
4,502,0024-7 б,,герметик акриловой цвет белый 7000 г,,,,"Герметик акриловой цвет Белый, 7 кг"
...,...,...,...,...,...,...,...,...
491,127,152-5,4680008143228.0,средство для уборки помещений после пожара с д...,средство для уборки помещений после пожара с д...,средство для уборки помещений после пожара с д...,средство для уборки помещений после пожара с д...,Средство для уборки помещений после пожара с д...
492,160,289-1,4680008147318.0,жидкое моющее средство для стирки шерсти шелка...,гель для стирки шерсти шелка и деликатных ткан...,гель для стирки шерсти шелка и деликатных ткан...,гель для стирки шерсти шелка и деликатных ткан...,"Жидкое моющее средство для стирки шерсти, шелк..."
493,74,192-05,4680008145413.0,средство для чистки гриля и духовых шкафовцоок...,цредство для чистки гриля и духовок цоокы грил...,цредство для чистки гриля и духовок цоокы грил...,цредство для чистки гриля и духовок цоокы грил...,Средство для чистки гриля и духовых шкафовCook...
494,34,186-5,4680008142733.0,средство для мытья полов с полимерным покрытие...,профессиональное средство для мытья полов с по...,профессиональное средство для мытья полов с по...,профессиональное средство для мытья полов с по...,Средство для мытья полов с полимерным покрытие...


In [ ]:
# тоже самое для теста

test['product_name'] = (test['product_name'].str.replace("\r\n", " ").str.lower().str.replace(r"[^a-zа-я0-9\s]", ""))
test['product_name']  = test['product_name'] .apply(replace_values_ml)
test['product_name']  = test['product_name'] .apply(replace_values_kg) # Замена значений в столбце 'объем' с помощью функции replace_values
test['product_name']  = test['product_name'] .map(transl)  # Добавление функции map для применения транслитерации к столбцу
#test['product_name']  = test['product_name'].apply(insert_color)

<ipython-input-20-14d55b1fa25d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test['product_name'] = (test['product_name'].str.replace("\r\n", " ").str.lower().str.replace(r"[^a-zа-я0-9\s]", ""))


In [ ]:
test

,product_id,product_name
0,12,средство универсальное просепт универсал спраы...
1,106,соль для посудомоечных машин просепт сплаш 150...
2,200,средство для мытья стекол и зеркал просепт опт...
3,38,концентрат просепт мултипоwер для мытья полов ...
4,403,удалитель ржавчины просепт руст ремовер 5000 м...
...,...,...
1685,267,антисептик для влажной древесины просепт био к...
1686,286,антисептик для бани и сауны просепт ецо сауна ...
1687,129,антиклей средство для удаления клея наклеек кл...
1688,1,антиклей средство для удаления клея наклеек кл...


# Fiass

Используем библиотеку Fiass, которая на GPU считает очень быстро. Так как я работаю на Colab, то у меня есть доступ к GPU. У заказчика надо смотреть. В данном случае я установила CPU версию.

In [ ]:
# Загружаем , модель, преобразуем датафрейм и загружаем индексы

processed_dataset = []
df=bert_mnb(df_product['name'])
d = df.shape[1]
index = faiss.IndexFlatL2(d)
index.add(df)
ranks = []


.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [ ]:
#Считаем ранг случайной записи из test.
#Выборка из теста - 50 случайных строк.

k=15
ranks=[]
for i in range(50):
    # Получаем ранги для каждого значения в обработанной строке
    j=random.randint(0, (len(df_product)-1))
    xq = np.array([bert_mnb(test.loc[j, 'product_name'])])
    D, I = index.search(xq, k)
    value= test.loc[j, 'product_id']
    indices=I[0,:] #df_product.loc[I[0,0], 'id']
    row_array = list(map(int, np.array(df_product['id'][indices]))) #list(map(int, string.strip("[]").split()))
    row_ranks = find_rank(row_array, value)
    ranks.append(row_ranks)
    print(row_array, 'Айди теста:', value, row_ranks)





[133, 493, 128, 321, 322, 328, 342, 219, 217, 341, 327, 332, 187, 456, 218] Айди теста: 133 1
[133, 493, 128, 321, 322, 328, 342, 219, 217, 341, 327, 332, 187, 456, 218] Айди теста: 133 1
[389, 387, 388, 413, 294, 382, 282, 272, 259, 30, 292, 276, 245, 417, 280] Айди теста: 389 1
[288, 289, 260, 281, 271, 412, 290, 182, 246, 275, 286, 416, 250, 190, 215] Айди теста: 288 1
[402, 401, 400, 399, 398, 405, 208, 392, 404, 397, 391, 254, 207, 246, 120] Айди теста: 402 1
[328, 327, 330, 219, 492, 333, 329, 218, 331, 321, 348, 345, 354, 272, 357] Айди теста: 328 1
[8, 189, 9, 188, 433, 259, 47, 30, 4, 66, 413, 282, 249, 187, 65] Айди теста: 8 1
[270, 272, 216, 282, 287, 294, 389, 269, 285, 276, 280, 242, 479, 292, 240] Айди теста: 282 4
[430, 432, 431, 427, 421, 429, 418, 423, 428, 424, 420, 422, 249, 426, 231] Айди теста: 430 1
[259, 262, 260, 261, 257, 263, 253, 285, 30, 258, 245, 189, 256, 255, 254] Айди теста: 259 1
[66, 65, 64, 69, 63, 286, 182, 217, 62, 20, 61, 190, 187, 264, 181] Айди т

### MRR

In [ ]:
#Считаем MRR

calculate_average(ranks)

0.7918571428571428

In [ ]:
report=pd.DataFrame({'Мethod': ['bert-base-nli-mean-tokens', 'bert-base-nli-mean-tokens/transl', 'bert-base-nli-mean-tokens/++', 'paraphrase-multilingual-mpnet-base-v2/tansl', 'paraphrase-multilingual-mpnet-base-v2', 'multilingual-e5-large'],
                         'MRR': [0.548, 0.45, 0.64, 0.657, 0.745, 0.791]})

report

,Мethod,MRR
0,bert-base-nli-mean-tokens,0.548
1,bert-base-nli-mean-tokens/transl,0.450
2,bert-base-nli-mean-tokens/++,0.640
3,paraphrase-multilingual-mpnet-base-v2/tansl,0.657
4,paraphrase-multilingual-mpnet-base-v2,0.745
5,multilingual-e5-large,0.790


В целом, полученный результат позвотяет чтитать, что модель в большинстве случаев находит нудный результат и выводит его первым.